<a href="https://colab.research.google.com/github/gaaloul8/projet-nosql/blob/main/Copie_de_Untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 4.2 MB/s eta 0:00:00 0:00:01


In [ ]:
import redis


In [ ]:
r = redis.Redis(host='redis', port=6379, decode_responses=True)


In [ ]:
nb_actors = r.keys("actor:*")
nb_movies = r.keys("movie:*")
print(f"Acteurs: {len(nb_actors)}, Films: {len(nb_movies)}")

Acteurs: 1319, Films: 922


In [ ]:
actors_before_1980 = []
for key in r.scan_iter("actor:*"):
    dob = r.hget(key, "date_of_birth")
    if dob and int(dob) < 1980:
        actors_before_1980.append(r.hmget(key, ["first_name", "last_name", "date_of_birth"]))
        if len(actors_before_1980) == 5:
            break
print(actors_before_1980)

[['Helen', 'Mirren', '1945'], ['Douglas', 'Henshall', '1965'], ['Embeth', 'Davidtz', '1965'], ['Jason', 'Flemyng', '1966'], ['John', 'Cullum', '1930']]


In [ ]:
for key in r.scan_iter("movie:*"):
    if r.hget(key, "title") == "The Imitation Game":
        result = r.hmget(key, ["genre", "rating"])
        print(f"Genre: {result[0]}, Note: {result[1]}")

Genre: Biography, Note: 8.1


In [ ]:
movies = []
for key in r.scan_iter("movie:*"):
    movies.append((r.hget(key, "title"), float(r.hget(key, "rating"))))
top_5 = sorted(movies, key=lambda x: x[1], reverse=True)[:5]
print(top_5)

[('Boy 9', 9.4), ('Vegas (doc)', 9.4), ('The Shawshank Redemption', 9.3), ('Ween Live in Chicago', 9.2), ('Over Canada: An Aerial Adventure', 9.1)]


In [ ]:
high_rated = sum(1 for key in r.scan_iter("movie:*") if float(r.hget(key, "rating")) > 7.5)
print(f"Films avec note > 7.5: {high_rated}")


Films avec note > 7.5: 183


In [ ]:
for key in r.scan_iter("movie:*"):
    if r.hget(key, "title") == "The Imitation Game":
        r.hset(key, "rating", 8.5)


In [ ]:
new_actor_id = len(r.keys("actor:*")) + 1
r.hset(f"actor:{new_actor_id}", mapping={
    "first_name": "Zendaya",
    "last_name": "",
    "date_of_birth": 1996
})

3

In [ ]:
for key in r.scan_iter("movie:*"):
    if r.hget(key, "title") == "The Room":
        r.delete(key)

In [ ]:
import redis

# Connexion à Redis
r = redis.Redis(host='redis', port=6379, decode_responses=True)

# 1. Compter les acteurs avec un nom de famille commençant par "P"
actor_count = 0
actors_with_p = []

for key in r.scan_iter("actor:*"):
    last_name = r.hget(key, "last_name")
    if last_name and last_name.startswith("P"):
        actor_count += 1
        actors_with_p.append(last_name)

print(f"Nombre d'acteurs avec un nom commençant par 'P': {actor_count}")

# 2. Films sortis après 2010 avec plus de 100,000 votes
recent_popular_movies = []

for key in r.scan_iter("movie:*"):
    release_year = r.hget(key, "release_year")
    votes = r.hget(key, "votes")

    if release_year and votes:
        if int(release_year) > 2010 and int(votes) > 100000:
            movie_data = {
                'title': r.hget(key, "title"),
                'release_year': release_year,
                'votes': votes,
                'rating': r.hget(key, "rating")
            }
            recent_popular_movies.append(movie_data)

print("\nFilms récents populaires:")
for movie in recent_popular_movies:
    print(f"{movie['title']} ({movie['release_year']}) - Votes: {movie['votes']}, Note: {movie['rating']}")

# 3. Créer des hashs top_movies_by_genre:<genre> avec le film le mieux noté par genre
genre_dict = {}

# D'abord collecter tous les films par genre
for key in r.scan_iter("movie:*"):
    genre = r.hget(key, "genre")
    rating = r.hget(key, "rating")
    title = r.hget(key, "title")

    if genre and rating and title:
        if genre not in genre_dict or float(rating) > float(genre_dict[genre]['rating']):
            genre_dict[genre] = {
                'title': title,
                'rating': rating
            }

# Ensuite créer les hashs Redis
for genre, data in genre_dict.items():
    r.hset(f"top_movies_by_genre:{genre}", mapping={
        'title': data['title'],
        'rating': data['rating'],
        'genre': genre
    })

# Vérification
print("\nTop films par genre:")
for genre_key in r.scan_iter("top_movies_by_genre:*"):
    print(f"{genre_key}: {r.hgetall(genre_key)}")

Nombre d'acteurs avec un nom commençant par 'P': 64

Films récents populaires:
San Andreas (2015) - Votes: 152249, Note: 6.1
The Jungle Book (2016) - Votes: 151342, Note: 7.6
Battle Los Angeles (2011) - Votes: 169988, Note: 5.7
Ted 2 (2015) - Votes: 123758, Note: 6.4
Pitch Perfect 2 (2015) - Votes: 101190, Note: 6.5
Whiplash (2014) - Votes: 667323, Note: 8.5
Room (2015) - Votes: 182004, Note: 8.2
Fifty Shades of Grey (2015) - Votes: 224710, Note: 4.1
The Interview (2014) - Votes: 251910, Note: 6.6
The Martian (2015) - Votes: 506641, Note: 8.0
Dawn of the Planet of the Apes (2014) - Votes: 323652, Note: 7.6
Non-Stop (2014) - Votes: 242556, Note: 6.9
300: Rise of an Empire (2014) - Votes: 229109, Note: 6.2
Need for Speed (2014) - Votes: 139153, Note: 6.5
Divergent (2014) - Votes: 412364, Note: 6.7
Everest (2015) - Votes: 142142, Note: 7.1
Mad Max: Fury Road (2015) - Votes: 578046, Note: 8.1
Gone Girl (2014) - Votes: 589116, Note: 8.1
Spy (2015) - Votes: 176403, Note: 7.1
Spotlight (2015)